In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lead
from pyspark.sql.types import (IntegerType, LongType)
from pyspark.sql.functions import (month, year)
from pyspark.sql.functions import udf, from_unixtime, unix_timestamp
#from pyspark import SparkContext

In [2]:
spark = SparkSession.builder.appName("Preprocessing").getOrCreate()

In [ ]:
raw_data_act = spark.read.parquet('ActivityObf.parquet')


In [3]:
raw_data_act = spark.read.parquet('sbObf.parquet')


In [4]:
from pyspark.sql.functions import rand

In [ ]:
df = raw_data_act.orderBy(rand()).select("monto_lineas_cred_disp")
df_pd = df.limit(500000)
df_pd.show(100)

In [6]:
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
hist(df_pd,10)

,monto_lineas_cred_disp
count,100.00000
mean,4506.10000
std,8833.78346
min,0.00000
25%,125.75000
50%,1172.00000
75%,3241.00000
max,49636.00000


In [ ]:
raw_data_act_int = raw_data_act.withColumn('user_id', raw_data_act['user_id'].cast(LongType()))

In [ ]:
df_act_wo_duplicates = raw_data_act_int.dropDuplicates()

In [ ]:
#df_act_wo_duplicates.write.csv("act_wo_duplicates.csv", header = True)

In [ ]:
df_act_wo_duplicates = spark.read.csv("act_wo_duplicates.csv/", header = True, inferSchema = True)


In [ ]:
#df_act_ordered = df_act_wo_duplicates.orderBy(['user_id', 'periodo'])

In [ ]:
from pyspark.sql import Window

w = Window.orderBy(['user_id','periodo']) ## --> esto ordena por user_id y periodo

# df_act_orderer_w_lead = df_act_ordered.withColumn("next_periodo", lead(df_act_ordered.periodo).over(w))\
#             .withColumn("next_user_id", lead(df_act_ordered.user_id).over(w))

df_act_orderer_w_lead = df_act_wo_duplicates.withColumn("next_periodo", lead(df_act_wo_duplicates.periodo).over(w))\
            .withColumn("next_user_id", lead(df_act_wo_duplicates.user_id).over(w))

In [ ]:
## Para merge de df's de act. y sb, ver --> https://stackoverflow.com/questions/38063657/pyspark-merge-outer-join-two-data-frames

In [ ]:

def estado_actividad(u_id, nxt_u_id, per, nxt_per):
    
    #def month_diff(x,y):
        
    #    return 12*(year(y)[0] - year(x)[0]) + (month(y)[0] - month(x)[0])
    
    if u_id == nxt_u_id and 12*(nxt_per.year - per.year) + (nxt_per.month - per.month) >= 4:#12*(year(nxt_per) - year(per)) + (month(nxt_per) - month(per)) >= 4:
        
        return 1
    else:
        return 0
    


In [ ]:
define_estados = udf(estado_actividad, IntegerType())    

# from_pattern = 'yyyy-MM-dd hh:mm:ss'
# to_pattern = 'yyyy-MM-dd'

# df_act_orderer_w_lead = df_act_orderer_w_lead.withColumn("periodo", from_unixtime(unix_timestamp(df_act_orderer_w_lead.periodo, 
#             from_pattern), to_pattern)).withColumn("next_periodo", 
#             from_unixtime(unix_timestamp(df_act_orderer_w_lead.next_periodo, from_pattern), to_pattern))
   

In [ ]:
df_act_final = df_act_orderer_w_lead.withColumn("estado", define_estados(df_act_orderer_w_lead.user_id,
                                                         df_act_orderer_w_lead.next_user_id,
                                                         df_act_orderer_w_lead.periodo,
                                                         df_act_orderer_w_lead.next_periodo)) 

In [ ]:
df_act_final.select(['user_id','periodo','estado']).write.csv("df_act_final.csv", header = True)

#### Dataframe act con estados

In [4]:
df_act_final= spark.read.csv('df_act_final.csv/', header = True, inferSchema = True)

In [5]:
from_pattern = 'yyyy-MM-dd hh:mm:ss'
to_pattern = 'yyyy-MM-dd'

df_act_final = df_act_final.withColumn("periodo", from_unixtime(unix_timestamp(df_act_final.periodo, 
            from_pattern), to_pattern))

In [6]:
#df_act_final.select(['user_id','periodo','estado']).show()
df_act_final.select('periodo').show()

+----------+
|   periodo|
+----------+
|2019-02-01|
|2019-03-01|
|2019-04-01|
|2019-05-01|
|2019-06-01|
|2019-07-01|
|2015-02-01|
|2015-03-01|
|2015-04-01|
|2015-05-01|
|2015-06-01|
|2015-07-01|
|2015-08-01|
|2015-09-01|
|2015-10-01|
|2015-11-01|
|2015-12-01|
|2016-01-01|
|2016-02-01|
|2016-03-01|
+----------+
only showing top 20 rows



#### Dataframe sb

In [7]:
df_sb = spark.read.parquet('sbObf.parquet')

In [ ]:
#df_sb.printSchema()

In [8]:
df_sb_int = df_sb.withColumn('user_id', df_sb['user_id'].cast(LongType()))
df_sb_int.printSchema()

root
 |-- fecha_proceso: date (nullable = true)
 |-- R04PERREF: string (nullable = true)
 |-- periodo: date (nullable = true)
 |-- user_id: long (nullable = true)
 |-- total_deudores_informados: integer (nullable = true)
 |-- deuda_vigente: long (nullable = true)
 |-- deuda_directa_morosa90: long (nullable = true)
 |-- deuda_directa_vencida: long (nullable = true)
 |-- deuda_directa_mora180: long (nullable = true)
 |-- deuda_indirecta_mora180: long (nullable = true)
 |-- deuda_directa_inversiones_financieras: long (nullable = true)
 |-- deuda_directa_ops_pacto: long (nullable = true)
 |-- deuda_indirecta_vigente: long (nullable = true)
 |-- deuda_indirecta_vencida: long (nullable = true)
 |-- deuda_directa_comercial: long (nullable = true)
 |-- deuda_directa_cred_consumo: long (nullable = true)
 |-- deuda_directa_hipotecaria: long (nullable = true)
 |-- deuda_directa_comercial_ext: long (nullable = true)
 |-- deuda_indirecta_comercial_ext: long (nullable = true)
 |-- deuda_directa_leas

In [ ]:
#df_sb_wo_duplicates = df_sb_int.drop_duplicates()

In [ ]:
df_sb_int.count()

In [ ]:
# df_sb_int.cache()
# df_act_final.cache()

In [9]:
df_sb_act = df_sb_int.orderBy(['user_id', 'periodo']).join(df_act_final.orderBy(['user_id', 'periodo']),
                                                           on = ['user_id', 'periodo'], how='inner' )

In [9]:
df_sb_act.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- periodo: date (nullable = true)
 |-- fecha_proceso: date (nullable = true)
 |-- R04PERREF: string (nullable = true)
 |-- total_deudores_informados: integer (nullable = true)
 |-- deuda_vigente: long (nullable = true)
 |-- deuda_directa_morosa90: long (nullable = true)
 |-- deuda_directa_vencida: long (nullable = true)
 |-- deuda_directa_mora180: long (nullable = true)
 |-- deuda_indirecta_mora180: long (nullable = true)
 |-- deuda_directa_inversiones_financieras: long (nullable = true)
 |-- deuda_directa_ops_pacto: long (nullable = true)
 |-- deuda_indirecta_vigente: long (nullable = true)
 |-- deuda_indirecta_vencida: long (nullable = true)
 |-- deuda_directa_comercial: long (nullable = true)
 |-- deuda_directa_cred_consumo: long (nullable = true)
 |-- deuda_directa_hipotecaria: long (nullable = true)
 |-- deuda_directa_comercial_ext: long (nullable = true)
 |-- deuda_indirecta_comercial_ext: long (nullable = true)
 |-- deuda_directa_leas

In [ ]:
df_sb_act.show()

In [11]:
df_sb_act.write.csv('dataFrame_actividad_features.csv', header = True)

KeyboardInterrupt: 

### ---------------------------------------------------------------------------------

### cosas de prueba

In [ ]:
df_sb_int.select('periodo').show()

In [ ]:
df_act_final.select('periodo').show()

In [ ]:
df_aux1 = spark.createDataFrame([(1, 15, 1), (2,30,0), (3,43,1), (4,21,0), (5,344,0)], ['col1','col2', 'col3'])
df_aux2 = spark.createDataFrame([(1, 15), (2,30), (3,43), (4,21), (7,89)], ['col1','col2'])



In [ ]:
df_aux1.show()

In [ ]:
df_aux2.show()

In [ ]:
new_df = df_aux2.join(df_aux1, on=['col1', 'col2'], how='inner')


In [ ]:
new_df.show()